In [2]:
import numpy as np

# Function to calculate the total distance of a route
def calculate_total_distance(route, distances):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distances[route[i]][route[i + 1]]
    total_distance += distances[route[-1]][route[0]]  # Return to the starting city
    return total_distance

# Function to generate an initial population of routes
def generate_initial_population(population_size, num_cities):
    population = []
    for _ in range(population_size):
        route = np.random.permutation(num_cities)
        population.append(route)
    return population

# Function to select parents using tournament selection
def tournament_selection(population, distances):
    tournament_size = 3
    candidates = np.random.choice(len(population), size=tournament_size, replace=False)
    best_candidate = min(candidates, key=lambda x: calculate_total_distance(population[x], distances))
    return population[best_candidate]

# Function to perform crossover (ordered crossover)
def crossover(parent1, parent2):
    child = [-1] * len(parent1)
    start, end = sorted(np.random.choice(len(parent1), size=2, replace=False))
    child[start:end] = parent1[start:end]
    
    remaining = [item for item in parent2 if item not in child]
    j = 0
    for i in range(len(child)):
        if child[i] == -1:
            child[i] = remaining[j]
            j += 1
    
    return np.array(child)

# Function to perform mutation (swap mutation)
def mutate(route):
    mutated_route = route.copy()
    idx1, idx2 = np.random.choice(len(route), size=2, replace=False)
    mutated_route[idx1], mutated_route[idx2] = mutated_route[idx2], mutated_route[idx1]
    return mutated_route

# Genetic Algorithm
def genetic_algorithm(distances, population_size=100, generations=1000):
    num_cities = len(distances)
    
    # Generate initial population
    population = generate_initial_population(population_size, num_cities)
    
    # Main evolution loop
    for generation in range(generations):
        # Select parents
        parents = [tournament_selection(population, distances) for _ in range(population_size)]
        
        # Create next generation
        next_generation = []
        for i in range(0, population_size, 2):
            parent1, parent2 = parents[i], parents[i + 1]
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            next_generation.extend([mutate(child1), mutate(child2)])
        
        population = next_generation
    
    # Find the best route in the final population
    best_route = min(population, key=lambda x: calculate_total_distance(x, distances))
    
    return best_route, calculate_total_distance(best_route, distances)

# Convert the provided data into a distance matrix
data = [
    13, 26, 27, 39, 0, 13, 19, 25, 31, 37, 43, 8,
    0, 10, 10, 10, 10, 5, 13, 19, 25, 31, 37, 43, 8,
    11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35,
    37, 39, 41, 42, 44, 45, 11, 15, 22, 23, 24, 26, 28,
    29, 9, 16, 20, 28, 30, 34, 40, 43, 47, 26, 31, 15,
    26, 29, 31, 15, 26, 29, 31, 38, 41, 5, 17, 31, 16,
    20, 30, 34, 22, 23, 32, 34, 35, 36, 22, 27, 6, 45,
    47, 25, 12, 25, 44, 45, 47, 6, 22, 11, 13, 16, 45,
    47, 12, 16, 20, 24, 29, 35, 39, 6, 21, 10, 32, 35,
    39, 10, 33, 37, 10, 41, 5, 17, 20, 24, 29, 34, 38,
    6, 27
]

num_cities = int(len(data) / 3)
distances = np.zeros((num_cities, num_cities))

for i in range(num_cities):
    for j in range(num_cities):
        x1, y1, _ = data[i * 3:i * 3 + 3]
        x2, y2, _ = data[j * 3:j * 3 + 3]
        distances[i][j] = np.linalg.norm(np.array([x1, y1]) - np.array([x2, y2]))

# Example usage
if __name__ == "__main__":
    best_route, best_distance = genetic_algorithm(distances)
    
    print("Best Route:", best_route)
    print("Best Distance:", best_distance)


Best Route: [24  1 10 37  2 20 30 16 27 11 36 31  0  7 17 29 33 39 41  4  5  8 35 25
 32 42 40 15  6 28 26 38 18 12 22 19 34  3 13 14 23 21  9]
Best Distance: 638.0556931949284
